In [123]:
import pandas as pd
import math 
import csv
import numpy as np
from fredapi import Fred
import json
import operator
import requests
import time
from ratelimit import limits, sleep_and_retry, RateLimitException

In [5]:
fred = Fred(api_key='597d1dc605851d73e7f227041c9dba3c')

In [13]:
#function that generates a list of time series IDs from the JSON page for FRED
def generateList():
    idList = []
    for i in range(100):
        r = requests.get('https://api.stlouisfed.org/fred/release/series?release_id='+ str(i) +'&api_key=597d1dc605851d73e7f227041c9dba3c&file_type=json')
        if 'seriess' in r.json():
            x = r.json()['seriess']
            for i in range(len(x)):
                idList.append(x[i]['id'])
    return idList

In [22]:
#generating a list for me to use
generated = generateList()

In [21]:
#using a sleep and retry descriptor to prevent exceeding rate limit
@sleep_and_retry
def getSeries(id):
    return fred.get_series(id)


In [71]:
#this function will make a list that takes values that only come from the intersection list. 
def makeList(intersect, ts):
    x = []
    for i in intersect:
        x.append(ts.at[i])
    return x

In [141]:
#this function will get datasets that have 100 or more intersecting date times from our generated list
#with whatever time series is inputted as 'id'. 
#IDs will input the generated list
#id will input the time series ID that we would like to compare to
def correlations100(id, IDs):
    compatibleIds = 0
    numID = 0
    correlations = {}
    x = getSeries(id)
    while compatibleIds < 100:
        y = getSeries(IDs[numID])
        intersectSet = set(x.index).intersection(set(y.index))
        if len(intersectSet) > 99:
            z = sorted(list(intersectSet))
            corr = np.corrcoef(makeList(z,x),makeList(z,y))[0][1]
            if np.isnan(corr) == False:
                correlations[IDs[numID]] = corr
                compatibleIds = compatibleIds + 1
        numID = numID +1
            
    return correlations
        

In [142]:
unsortedDict = correlations100('GDP', generated)


In [125]:
#sorts my outputted dictionary by value

sorted_d = sorted(unsortedDict.items(), key=operator.itemgetter(1))

In [138]:
# creates dataframe out of my dictionary and renames it to include columns
df = pd.DataFrame.from_dict(sorted_d)
df2 = df.rename(columns = {0:'ID',1:'Correlation'})
df2

,ID,Correlation
0,RSDSELD,-0.728782
1,RSDSELDN,-0.633747
2,MARTSMPCSM441XUSN,-0.213301
3,MARTSMPCSM441USN,-0.211170
4,MARTSMPCSM451USS,-0.167572
...,...,...
95,RSHPCSN,0.995603
96,RSHPCS,0.996331
97,RSFSXMV,0.997490
98,MARTSSM44W72USS,0.998976


In [140]:
#converts my dataframe to a csv file in a provided file path.
df2.to_csv('/Users/justinhsiang/Desktop/100correlations/correlations to gdp.csv')